![](https://raw.githubusercontent.com/mossaybo/colab-book-ocr/main/banner.png)

In [ ]:
#Setting up Variables #
original_pdf = "original.pdf"
output_pdf = "result_ocr.pdf"
lang_code = "ara"   #<-- default to Arabic, other supported languages: https://tinyurl.com/tesslangcodes

# optional , leave empty to ocr all the book, or set BOTH to ocr a specific part.
first_page = ""
last_page = ""

In [ ]:
# Upload the Book to files (give it same name as original_pdf) or Download it directly: uncomment and change URL #
# !wget "http://www.almeshkat.net/books/archive/books/hewar12.pdf" -O $original_pdf

In [ ]:
%%capture
# Setting up tools #
!apt install tesseract-ocr tesseract-ocr-$lang_code poppler-utils
!pip install PyPDF2

langs = !tesseract --list-langs
class LanguageNotInstalled(Exception):
  pass
if lang_code not in langs:
  raise LanguageNotInstalled("The Language", lang_code, "wasn't Installed correctly, please re-check lang_code and insure it's available")

from PyPDF2 import PdfFileMerger
from IPython.utils import io
from IPython.display import clear_output

In [ ]:
# original pdf ----> images #
%cd /content/
%mkdir temp
%cd temp
if first_page:
  !pdfimages -f $first_page -l $last_page /content/$original_pdf page
else:
  !pdfimages /content/$original_pdf page

In [ ]:
# images -----> pdfs #
pages_img = !ls -1
def ocr_core(image, lang_code):
  page_stem= image.rsplit('.')[0]
  !tesseract $image -l $lang_code $page_stem pdf

for image in pages_img:
  num = int(''.join([i for i in image if i.isdigit()])) + 1
  all = len(pages_img)

  clear_output()
  print("Working on Page :", num, "/", all)
  print("Progress: ", int(num / all * 100), "%")
  
  #supress tesseract verbose output
  with io.capture_output() as captured:
    ocr_core(image, lang_code)


print("PDFs are ready , Ready for merge")

In [ ]:
# pdfs ---merge---> output.pdf
pdfs = !ls -1 *.pdf
merger = PdfFileMerger()

for pdf in pdfs:
  try:
    merger.append(pdf)
  except:
    print("error in "+ pdf + ", skipping it ...")
    continue

merger.write("/content/"+output_pdf)
merger.close()
print("\nDONE, output file :", output_pdf)